# 1 -Get the data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d wanderdust/skin-lesion-analysis-toward-melanoma-detection

In [ ]:
! unzip /content/skin-lesion-analysis-toward-melanoma-detection.zip

# 2 - Explore the data and prepare it

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import pandas as pd
import numpy as  np
import tensorflow as tf
from tensorflow import keras
import os 
import cv2
import PIL
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau

In [ ]:
Lables = [ 'nevus', 'seborrheic_keratosis' ,'melanoma' ]

In [ ]:
train_dir = '/content/skin-lesions/train'
test_dir = '/content/skin-lesions/test'
val_dir = '/content/skin-lesions/valid'

In [ ]:
def GetDatasetSize(path):
    num_of_image = {}
    for folder in os.listdir(path):
        # Counting the Number of Files in the Folder
        num_of_image[folder] = len(os.listdir(os.path.join(path, folder)));
    return num_of_image;
    
train_set = GetDatasetSize(train_dir)
val_set = GetDatasetSize(val_dir)
test_set = GetDatasetSize(test_dir)
print(train_set,"\n\n",val_set,"\n\n",test_set)

In [ ]:
labels = list(train_set.keys())
train_list = list(train_set.values())
val_list = list(val_set.values())
test_list = list(test_set.values())

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width, train_list, width, label='Train')
rects2 = ax.bar(x, val_list, width, label='Val')
rects3 = ax.bar(x + width, test_list, width, label='Test')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Images Count')
ax.set_title('Dataset')
ax.set_xticks(x, labels)
plt.xticks(rotation=15)
ax.legend()


fig.tight_layout()

plt.show()

In [ ]:
def Get_data(train_dir , test_dir , val_dir ):
  x_train =[]
  y_train =[]
  x_test = []
  y_test = []
  x_val =  []
  y_val =  []
  train_dir_list = os.listdir(train_dir)
  for i in range(len(train_dir_list)):
    sub_dir = os.path.join(train_dir , Lables[i])
    sub_dir_list = os.listdir(sub_dir)
    for j in range(len(sub_dir_list)):
       if str(sub_dir_list[j]).split('.')[1] =='jpg' :
         x_train.append(os.path.join(sub_dir , sub_dir_list[j]))
         y_train.append(i)
  test_dir_list = os.listdir(test_dir)    
  for i in range(len(test_dir_list)):
    sub_dir = os.path.join(test_dir , Lables[i])
    sub_dir_list = os.listdir(sub_dir)
    for j in range(len(sub_dir_list)):
      if str(sub_dir_list[j]).split('.')[1] =='jpg' :
         x_test.append(os.path.join(sub_dir , sub_dir_list[j]))
         y_test.append(i) 
  val_dir_list = os.listdir(val_dir)    
  for i in range(len(val_dir_list)):
    sub_dir = os.path.join(val_dir , Lables[i])
    sub_dir_list = os.listdir(sub_dir)
    for j in range(len(sub_dir_list)):
       if str(sub_dir_list[j]).split('.')[1] =='jpg' :
         x_val.append(os.path.join(sub_dir , sub_dir_list[j]))
         y_val.append(i) 
  return np.array(x_train) ,np.array(x_test) , np.array(x_val) , np.array(y_train) , np.array(y_test) , np.array(y_val)


In [ ]:
x_train ,x_test , x_val , y_train , y_test , y_val = Get_data(train_dir,test_dir,val_dir)

In [ ]:
print(y_train.shape==x_train.shape)
print(y_test.shape==x_test.shape)
print(y_val.shape==x_val.shape)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

In [ ]:
# we will see data split counts
plt.bar(['train' , 'test' ,'val'] , height =[len(x_train) , len(x_test) ,len(x_val)]
         )
plt.xlabel('data_')
plt.ylabel('Length')
plt.show()

In [ ]:
def image_show(index):
  image = np.array(cv2.imread(x_train[index]))
  print(image.shape)
  plt.imshow(image)
  plt.title(Lables[y_train[index]])
  plt.show() 

In [ ]:
image_show(1700)

In [ ]:
def image_show_resized(index , gray):
  if gray :
    image = image = cv2.imread(x_train[index] , cv2.IMREAD_GRAYSCALE)
  else :
    image = cv2.imread(x_train[index] )
  image = cv2.resize(image , (150,150))
  image = np.array(image)
  print(image.shape)
  plt.imshow(image)
  plt.title(Lables[y_train[index]])
  plt.show()

In [ ]:
image_show_resized(1700 , gray = False)

In [ ]:
# we will see data distribution
def plot_data_distribution(y_train):
  unique, counts = np.unique(y_train, return_counts=True)
  plt.bar([Lables[i] for i in unique] , height = counts , color ='green')
  plt.xlabel('data_unique')
  plt.xlabel('data_frequency')
  plt.show()
  for i in range(len(unique)):
    print(Lables[i] ,':' , counts[i])

In [ ]:
plot_data_distribution(y_train)

# 3 - prepare the data for training and build data Generator

In [ ]:
x_train[0]

In [ ]:
def image_loaders(image_paths):
  images =[]
  for i in range(len(image_paths)):
    image = cv2.imread(image_paths[i] )
    image = cv2.resize(image , (150,150))
    image = np.array(image)
    images.append(image)
  images = np.array(images)
  images = images/255
  return images

In [ ]:
x_train = image_loaders(x_train)
x_val = image_loaders(x_val)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

In [ ]:
from tensorflow.keras.preprocessing import image
train_datagen = image.ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2 , rescale = 1./255 , horizontal_flip = True, vertical_flip = True)
val_datagen = image.ImageDataGenerator(rescale = 1./255)
test_datagen = image.ImageDataGenerator(rescale = 1./255)

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# we need to shuffle the data
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, to_categorical(y_train), random_state=20)
x_test, y_test = shuffle(x_test, to_categorical(y_test), random_state=20)
x_val, y_val = shuffle(x_val,to_categorical(y_val), random_state=20)

# 4 - build the model and compute loss function

In [ ]:
# build_model
model = Sequential()
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,3)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 3 , activation = 'softmax'))
model.compile(optimizer = "adam" , loss = keras.losses.CategoricalCrossentropy() , metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model 
from tensorflow.keras import layers 
from tensorflow.keras import Model
# Adding Model check point Callback
mc = ModelCheckpoint(filepath="oc_cnn_best_model.hdf5",
                     verbose= 0,
                     save_best_only= True
                     );

In [ ]:
model.summary()

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)
history1 = model.fit(train_datagen.flow(x_train , y_train) 
          ,epochs=5
          ,verbose=1,
          validation_data=val_datagen.flow(x_val , y_val),
          callbacks=[learning_rate_reduction ,mc]
          )

In [ ]:
x_test = image_loaders(x_test)

In [ ]:
# Checking the Accuracy of the Model 
accuracy_cnn = model.evaluate(x_test , y_test)[1] 
print(f"The accuracy of the model is = {accuracy_cnn*100} %")

In [ ]:
# Plot model performance
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs_range = range(1, len(history1.epoch) + 1)

plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Set')
plt.plot(epochs_range, val_acc, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Set')
plt.plot(epochs_range, val_loss, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')

plt.tight_layout()
plt.show()

vgg-16 model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(
    weights='imagenet',
    include_top=False, 
    input_shape=(150,150,3)
)

In [ ]:
layers =  base_model.get_layer('vgg16').layers


In [ ]:
NUM_CLASSES = 3

vgg_model = Sequential()
vgg_model.add(base_model)
vgg_model.add(layers.Flatten())
vgg_model.add(layers.Dropout(0.25))
vgg_model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

vgg_model.layers[0].trainable = False

vgg_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
vgg_model.summary()

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)
history2 = vgg_model.fit(train_datagen.flow(x_train , y_train) 
          ,epochs=1
          ,verbose=1,
          validation_data=val_datagen.flow(x_val , y_val),
          callbacks=[learning_rate_reduction ,mc]
          )

In [ ]:
# Checking the Accuracy of the Model 
accuracy_vgg = vgg_model.evaluate(x_test , y_test)[1] 
print(f"The accuracy of the model is = {accuracy_vgg*100} %")

In [ ]:
# Plot model performance
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']
epochs_range = range(1, len(history2.epoch) + 1)

plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Set')
plt.plot(epochs_range, val_acc, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Set')
plt.plot(epochs_range, val_loss, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')

plt.tight_layout()
plt.show()

In [ ]:
algos = ['CNN', 'VGG16']
accuracy = [accuracy_cnn, accuracy_vgg]
accuracy = np.floor([i * 100 for i in accuracy])
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(algos, accuracy, color ='red', width = 0.3)
 
plt.xlabel("Algorithms Applied")
plt.ylabel("Accuracy")
plt.show()

gradient-cam

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import imagenet_utils

In [ ]:
orig = cv2.imread('//content/skin-lesions/test/nevus/ISIC_0012149.jpg')
resized = cv2.resize(orig, (150, 150))

In [ ]:
resized.shape

In [ ]:
plt.imshow(resized)

In [ ]:
image = np.expand_dims(np.array(resized)/255,axis = 0)

In [ ]:
preds = model.predict(image)
i = np.argmax(preds[0])

In [ ]:
def compute_cam(model , layername , orig ):
  gradModel = Model(inputs=[model.inputs], outputs= [model.get_layer(layername).output, model.output])
  with tf.GradientTape() as tape:
     inputs = tf.cast(image, tf.float32)
     (convOutputs, predictions) = gradModel(inputs)
     loss = predictions[:, i]
  grads = tape.gradient(loss, convOutputs)
  castConvOutputs = tf.cast(convOutputs > 0, "float32")
  castGrads = tf.cast(grads > 0, "float32")
  guidedGrads = castConvOutputs * castGrads * grads
  convOutputs = convOutputs[0]
  guidedGrads = guidedGrads[0]
  weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
  cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)
  (w, h) = (image.shape[2], image.shape[1])
  heatmap = cv2.resize(cam.numpy(), (w, h))
  numer = heatmap - np.min(heatmap)
  denom = (heatmap.max() - heatmap.min()) + 1e-8
  heatmap = numer / denom
  heatmap = (heatmap * 255).astype("uint8")
  # then we will overlay the image with heatmap
  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
  output = cv2.addWeighted(orig, 0.5, heatmap, 0.5, 0)
  return heatmap , output

In [ ]:
heatmap , output = compute_cam(model ,'conv2d_4' , resized )

In [ ]:
import imutils
cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1)
# display the original image and resulting heatmap and output image
# to our screen
output = np.vstack([resized, heatmap, output])
output = imutils.resize(output, height=700)
from google.colab.patches import cv2_imshow
cv2_imshow(output)
cv2.waitKey(0)
cv2.destroyAllWindows()